# Portal Tasks and the ArcGIS API for Python

Portal for ArcGIS is a big interface for and facilitator of GIS users. It provides an interface for users to interact with a bunch of server components that get stitched together in one big federation of glory. The default and primary way we interact with it is the user interface--like one big website or application. I've noticed a lot of organizations with Portal for ArcGIS haven't yet explored the alternative routes to carry out tasks.

This post is a short demonstration of one of those alternative routes Esri provides: the ArcGIS API for Python. The API has grown to encompass a whole lot more than I can cover in one post, or even a lot of posts, so this will be a quick but useful working demo for interrogating some components of your Portal. Also covered is a simple method to upload PDFs from a folder on disk to your Portal, share those PDFs, and build links to them for your organization's users to consume.

All of the demo sections will work equally well between Portal and ArcGIS online except for the PDF access. ArcGIS Online hosts this kind of content through Amazon and therefore building a direct link and gaining access is a different and not covered here.

In [ ]:
# We start by importing our python packages. All of these packages should come with your python installation
# assuming you have at least python 3.5. If you have ArcGIS pro installed you will already have a sufficient
# version of python. The API provides good access so long as you have at least Portal 10.5. I haven't tested
# compatibility but I would suspect these simple operations to work if you have this version or higher.

import os
import time
import arcgis
from arcgis.gis import GIS

# IPython is not required to interact with Portal, it is included here to use the display class while in Jupyter
# Notebooks which is a web application that lets you run interactive python code.

from IPython.display import display

In [ ]:
# The username and password variables are the values you use to log into your Portal (or AGOL).

username = 'YOURUSERNAME'
password = 'YOURPASSWORD'

In [ ]:
# For Portal access, the URL will either need to be the Web Adaptor URL used to access the environment through web tier
# authentication such that users have access to Portal from the public web OR you will need to be on your network (or
# connected through a VPN) if your Portal is not accessible from the public web. Most of the time the URL will be the URL
# you use to get to the main Portal site. If working with AGOL, use the AGOL URL like https://COMPANY.maps.arcgis.com

# This object provides helper objects to do all the things we want, so we build this first to get access to the organization.

gis = GIS(r'YOURPORTALURL', username, password, verify_cert=False)

In [ ]:
# Here we run a search and get a list of all users using a wildcard.

users = gis.users.search('*')

# This statement prints the number of users in the list, ours is up to 100 now!

print(len(users))

In [ ]:
# This statement prints the users one-by-one.

for u in users:
    print(u)

In [ ]:
# This variable will serve as the name of a group we will create (all without a UI!).

groupname = 'DELETEME'

In [ ]:
# The API is nicely organized, so "gis.groups.create" does what you might expect. We just need to supply it with some
# properties like a title, tags, etc. We can also set the access level right at this point of creation.

print('creating test group')
temp_group = gis.groups.create(title=groupname,
                              tags='delete',
                              description='Temporary group',
                              snippet='placeholder description',
                              access='org',
                              is_invitation_only='False')

In [ ]:
# Now that we have created a group, lets search for and create a list of groups with a query. We can also go the the Portal
# Group section to find the group in the list.

group_search = gis.groups.search("title:{} AND owner:{}".format(groupname, username), outside_org = False)

In [ ]:
# We will print out the groups found in our search to make sure we have just our new group selected.

for group in group_search:
    print('group name: ' + str(group.title))
    print('group access: ' + str(group.access))
    print('group members: ' + str(group.get_members()))

In [ ]:
# There was only one group in our list, so lets select that item and use it to send a delete command.

group = group_search[0]

print('deleting group: ' + str(group.title))
group.delete()

# And that is all it takes, the group is gone!

In [ ]:
# Its not uncommon to get a request to use Portal as a nice organizational storage and access point for things like PDFs
# that commonly accompany our GIS data. Let's start with a folder of PDFs we want to host on Portal.

pdf_dir = r'C:\pdf'
pdfs = []

In [ ]:
# Using the walk function in our os package lets us build a list of all the files in the pdf folder, which all happen to
# be the PDFs we want to upload. We can confirm by printing out that list.

for root, dirs, files in os.walk(pdf_dir):
    for filename in files:
        pdfs.append(os.path.join(root, filename))

print(pdfs)

In [ ]:
# Now we will create another helper object called ContentManager and we will name it manager.

manager = arcgis.gis.ContentManager(gis)

In [ ]:
# Since we have a list of PDF files with a path to their location on disk, we can go through that list one at a time to add
# each file to Portal.

for pdf in pdfs:
    pdf_name = os.path.splitext(os.path.basename(pdf))[0]
    pdf_properties = {'type':'PDF', 'description':'PDF upload for map viewer','title':pdf_name, 'tags':'pdf', 'access':'org'}
    manager.add(pdf_properties, data=pdf)
    print('adding PDF: ' + pdf_name)

In [ ]:
# Those PDFs should be in Portal. On AGOL the PDFs should show up in Content. Depending on the version of Portal they may 
# or may not be easily found in the user interface of Portal. But they are there! Lets run a search to find confirm our
# PDF's made it.

pdf_search = manager.search(query="owner:{}".format(username), item_type='PDF')

In [ ]:
# Now we can print the list and see that yes, they are on Portal.

for pdf in pdf_search:
    print(pdf)

In [ ]:
# But how can user's access these PDFs? Lets change how they are shared and build a URL link that we can drop in a field
# of some feature in our GIS, or add to a document, or anything else we want to do. Remember that these will be accessible
# according to how you share them with the function share below and so long as the users have access to Portal, such as
# being on the same network or having access to Portal regularly. There are some authentication schemes that may prohibit
# this kind of direct access. If all goes to plan here a list of click-able links will get spit out for you to test.

portal_url_base = r'https://YOURDOMAIN/portal/sharing/rest/content/items/'

for pdf in pdf_search:
    pdf.share(everyone=True)
    pdf_item = gis.content.get(pdf.id)
    pdf_url = portal_url_base + pdf.id + r'/data'
    print(pdf_url)
    time.sleep(1)

In [ ]:
# To clean these up for our demonstration, we will get a list of what we've just added. Confirm this list only contains
# the items we want to now get rid of.

pdf_search = manager.search(query="owner:{}".format(username), item_type='PDF')

In [ ]:
# Finally, we run a delete operation to nuke the PDFs.

for pdf in pdf_search:
    print('delete: ' + str(pdf.id))
    pdf.delete()

I hope this quick demo of some useful ArcGIS API for Python functions gives you an idea of how you can access your Portal in more programmatic ways. You can visit find a whole lot of samples at the github page here https://github.com/Esri/arcgis-python-api/releases/tag/v1.8.2 and you can dig into the depth of operations available in the other Esri products enabled through AGOL or the base deployment including Portal at the API reference page here https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html.

As you can imagine these tools can make a big impact on adminstration of your overall GIS. It can also be used by regular GIS users to perform analysis and conduct data operations needed for various tasks and projects. Many of these use cases are sampled at the previously linked github page. You can also schedule scripts built with the ArcGIS API for Python using Esri's ArcGIS Notebook Server. For more info you can read up on it here https://enterprise.arcgis.com/en/notebook/latest/install/windows/introduction-to-arcgis-notebook-server.htm. 

I think ArcGIS Notebook Server is a topic and product that will get heavy use as time goes on and as organizations adopt it, so I will certainly write about it in the future. For now, think about how this alternative approach to conducting tasks in Portal might streamline your workflow or help you automate a routine task and free up some more time for the important things in life like studying UFOs or estimating the percentage of feldspar in various types of sand.